# BACKPROJECT IMAGE CLASSIFICATION RESULTS ONTO THE POINT CLOUD

In [5]:
from rdflib import Graph, URIRef
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import random as rd
import pandas as pd
import cv2
import laspy
import copy
from matplotlib import pyplot as plt
import math
import pathlib
import PIL
import PIL.Image
# import tensorflow as tf
# from tensorflow import keras
import random
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.tools import progresstools as pt
from geomapi.utils import imageutils as iu
from geomapi.tools import machinelearningtools as mlt

import geomapi.tools as tl

In [6]:
%load_ext autoreload

In [7]:
%autoreload 2

## USER INPUT

general

In [8]:
projectPath= os.path.join("D:\\Data\\2023-01 Paestum")
sessionPath = os.path.join(projectPath,"Research")

test1

In [12]:
#PCD
lasPath=os.path.join(projectPath,"PCD","dense_point_cloud_0.004.las")
outputlasPath=os.path.join(projectPath,"PCD","paestum_test1_out.las")
transform=np.array([[1.0,0.0, 0.0,  5.3857862609899980e+000], 
                [0.0, 1.0, 0.0, 2.5782303777102851e+002],
                [0.0, 0.0, 1.0 ,-6.0074548600459288e+000],
                [0.0 ,0.0, 0.0, 1.000000000000]]) # -> apply to pcd
scale=1/1.8781217664810130e+000 # -> apply to pcd

# -10
# +10
#  5.3857862609899980e+000
# 2.5782303777102851e+002 
#Images
xmlPath=os.path.join(projectPath,'IMG','cameras.xml')
sensorid=1
focalLength35mm= 2428.49184473261
rotation=np.array([[-9.9927572727173009e-001, -9.2989357887395263e-003 ,3.6899196180300041e-002,0],
                        [ 3.4752208827689118e-002, 1.7201294833769601e-001, 9.8448150291702796e-001,0],
                        [-1.5501769807091738e-002, 9.8505079838421061e-001 , -1.7156520549190035e-001,0],
                        [0,0,0,1]]) # -> apply to images
       
#classification
# modelMPath=os.path.join(sessionPath,"Facade_classification","Model_MobileNetV2__materials")
# modelTCPath=os.path.join(sessionPath,"Facade_classification","Model_MobileNetV2_BuildingTechniques")
# modelDPath=os.path.join(sessionPath,"Facade_classification","Model_MobileNetV2_Damages")
# trainingsfolder=pathlib.Path(os.path.join(sessionPath,'shapeConv','temp'))
# tf_image_folder=pathlib.Path(os.path.join(sessionPath,'Classification_test1','im_temp'))
# tf_mask_folder=pathlib.Path(os.path.join(sessionPath,'Classification_test1','mask_temp'))
output_dir=os.path.join(sessionPath,'shapeConv')
dir_img=os.path.join(output_dir,'image') # already full
dir_class=os.path.join(output_dir,'class') # already full
dir_depth=os.path.join(output_dir,'depth') #already full
dir_hha=os.path.join(output_dir,'hha') 
dir_labels=os.path.join(output_dir,'label12')

# tempfolder=pathlib.Path(os.path.join(sessionPath,'Facade_classification','temp'))
# batch_size = 32 #! same as model
# class_names_m=[0.0,1.0]
# class_names_tc=[0.0,1.0,2.0]
# class_names_d=[1.0,2.0]
# imagePath=os.path.join(projectPath,'Research')
# filteringWeight=3
# colors=[(255,0,0),(0,255,0),(0,0,255)]
# pixelRange=224
# rayOffsetDistance=0.2
skip=300 #nth file tu use


## PREPROCESS POINT CLOUD

Create point cloud node with proper offset.

In [6]:
las  = laspy.read(lasPath)
pcdNode=PointCloudNode(name=ut.get_filename(lasPath),
                        lasPath=lasPath,
                        resource=gmu.las_to_pcd(las,transform=transform)
                        )
pcdNode.resource.scale(scale,(0,0,0)) #pcdNode.resource.get_center()

PointCloud with 2785923 points.

## PREPROCESS THE IMAGES WITH THE POINT CLOUD

Create imageNodes with proper offset.

In [11]:
imgNodes=tl.img_xml_to_nodes(xmlPath,getResource=False)
for n in imgNodes:
    n.cartesianTransform=rotation @ n.cartesianTransform 

meshes=[gmu.generate_visual_cone_from_image(n.cartesianTransform, height =0.4) for n in imgNodes]
print(str(len(imgNodes))+ " Nodes created!")

894 Nodes created!


In [11]:
#get list of files
files=ut.get_list_of_files(ut.get_folder(xmlPath))
files=[f for f in files if f.endswith('.JPG')]

#select every fifth files
imgNodes=[]
for f in files[0::skip]:    
    imgNodes.append(ImageNode(name=ut.get_filename(f),
              path=f,
              xmlPath=xmlPath,
                focalLength35mm=focalLength35mm))
# apply transform to put imagery in local coordinates
for n in imgNodes:
    n.cartesianTransform=rotation @ n.cartesianTransform 
       
meshes=[gmu.generate_visual_cone_from_image(n.cartesianTransform, height =0.4) for n in imgNodes]
print(str(len(imgNodes))+ " Nodes created!")

3 Nodes created!


In [8]:
{key:value for key, value in imgNodes[0].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_xmlPath': 'D:\\Data\\2023-01 Paestum\\IMG\\cameras.xml',
 '_xmpPath': None,
 '_orientedBoundingBox': None,
 'imageWidth': 6048,
 'imageHeight': 4032,
 'focalLength35mm': 2428.49184473261,
 'keypoints': None,
 'descriptors': None,
 '_subject': rdflib.term.URIRef('file:///DSC_4226'),
 '_graph': None,
 '_graphPath': None,
 '_path': 'D:/Data/2023-01 Paestum/IMG/DSC_4226.JPG',
 '_name': 'DSC_4226',
 '_timestamp': '2014-06-06T17:21:42',
 '_resource': None,
 '_cartesianTransform': array([[  0.01816121,   0.99750693,  -0.06819158,   8.0859977 ],
        [ -0.42725536,   0.06940471,   0.90146317, -11.42426073],
        [  0.90394857,   0.01276355,   0.42745066,  -2.17309796],
        [  0.        ,   0.        ,   0.        ,   1.        ]]),
 'resolutionUnit': 2,
 'geospatialTransform': [None, None, None],
 'coordinateSystem': 'geospatial-wgs84'}

visualize the inputs

In [24]:
joinedImages=gmu.join_geometries(meshes)
joinedImages=joinedImages.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([joinedImages]+[pcdNode.resource])

colorize pcd per class

In [25]:
labels=las['Scalar field #15']
values=np.unique(labels)
pcd=copy.deepcopy(pcdNode.resource)
colors=np.array([ut.random_color() for v in range(len(values))])
colorArray=gmu.array_to_colors(labels,colors)
pcd.colors=o3d.utility.Vector3dVector(colorArray)

In [27]:
# optional
octree=pt.pcd_to_octree(pcd,8) #if octree is None else octree
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh

In [30]:
o3d.visualization.draw_geometries([joinedImages]+[mesh])


(optional) Get depth and class labels image per image from the point cloud

In [ ]:
# colorImages1,depthImages1=pt.project_pcd_to_rgbd_images(pcd,myImageNodes,depth_max=30,fill_black=40)#
# for c,d,n in zip(colorImages1,depthImages1,imgNodes): 
#     n.mask1=pt.remap_color_images_to_masks(c,colors)[0]
#     n.colorImage1=c 
#     n.depthImage1=d 

Get depth and class labels image per image from the octree

In [32]:
octree=pt.pcd_to_octree(pcd,8) #if octree is None else octree
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh
# colorImages2,depthImages2=pt.project_meshes_to_rgbd_images(mesh, imgNodes) # no depth filtering currently
width=640
height=480
extrinsics=[n.cartesianTransform for n in imgNodes]
intrinsics=[np.array([[focalLength35mm/(n.imageWidth/width),0,width/2-0.5],[0,focalLength35mm/(n.imageHeight/height),height/2-0.5],[0,0,1]]) for n in imgNodes]
colorImages2,depthImages2=gmu.project_meshes_to_rgbd_images(mesh, extrinsics,intrinsics) # no depth filtering currently
for c,d,n in zip(colorImages2,depthImages2,imgNodes): 
    n.resource=iu.image_resize(n.resource,width=width,height=height)
    n.mask2=pt.remap_color_images_to_masks(c,colors)[0]
    n.colorImage2=c 
    n.depthImage2=d 
    n.hha=mlt.depth_map_to_hha(intrinsics[0],d,d) #! this takes 9min per image , perhaps scale it down and up
    
    # n.hha=mlt.depth_map_to_hha(n.get_intrinsic_camera_parameters().intrinsic_matrix,n.depthImage2,n.depthImage2) #! this takes 9min per image , perhaps scale it down and up

d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:157: RuntimeWarning: invalid value encountered in true_divide
  X_Z = np.divide(X, Z)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:158: RuntimeWarning: invalid value encountered in true_divide
  Y_Z = np.divide(Y, Z)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:162: RuntimeWarning: invalid value encountered in true_divide
  X_ZZ = np.expand_dims(np.divide(X, ZZ), axis=2)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:163: RuntimeWarning: invalid value encountered in true_divide
  Y_ZZ = np.expand_dims(np.divide(Y, ZZ), axis=2)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:184: RuntimeWarning: divide by zero encountered in true_divide
  b = np.divide(-detAtA, divide_fac)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:184: RuntimeWarnin

In [13]:
# subdivide images to 640,480
octree=pt.pcd_to_octree(pcd,8) #if octree is None else octree
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh
colorImages2,depthImages2=pt.project_meshes_to_rgbd_images(mesh, imgNodes) # no depth filtering currently
width=640
height=480
imgTrainingNodes=[]
for c,d,n in zip(colorImages2,depthImages2,imgNodes): 
    imgList,roiList=iu.subdivide_image(n.resource,m=3,n=3,includeLast=True) 
    imgList=[iu.image_resize(img, width=640,height=480) for img in imgList ]
#     n.resource=iu.image_resize(n.resource,width=width,height=height)
    masks,_=iu.subdivide_image(pt.remap_color_images_to_masks(c,colors)[0],m=3,n=3,includeLast=True) 
    masks=[iu.image_resize(img, width=640,height=480) for img in masks ]
    vizs,_=iu.subdivide_image(c ,m=3,n=3,includeLast=True) 
    vizs=[iu.image_resize(img, width=640,height=480) for img in vizs ]
    
    depths,_=iu.subdivide_image(d ,m=3,n=3,includeLast=True) 
    depths=[iu.image_resize(img, width=640,height=480) for img in depths ]
    
    hhas,_=iu.subdivide_image(mlt.depth_map_to_hha(n.get_intrinsic_camera_parameters().intrinsic_matrix,d,d),m=3,n=3,includeLast=True)  #! this takes 9min per image , perhaps scale it down and up
    hhas=[iu.image_resize(img, width=640,height=480) for img in hhas ]
    
    for img,roi,mask,viz,depth,hha in zip(imgList,roiList,masks,vizs,depths,hhas):
        imgTrainingNodes.append(ImageNode(resource=img,
                                isDerivedFrom=n.subject,
                                roi=roi,
                                mask=mask,
                                color=viz,
                                depth=depth,
                                hha=hha))  

d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:157: RuntimeWarning: invalid value encountered in true_divide
  X_Z = np.divide(X, Z)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:158: RuntimeWarning: invalid value encountered in true_divide
  Y_Z = np.divide(Y, Z)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:162: RuntimeWarning: invalid value encountered in true_divide
  X_ZZ = np.expand_dims(np.divide(X, ZZ), axis=2)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:163: RuntimeWarning: invalid value encountered in true_divide
  Y_ZZ = np.expand_dims(np.divide(Y, ZZ), axis=2)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:184: RuntimeWarning: divide by zero encountered in true_divide
  b = np.divide(-detAtA, divide_fac)
d:\Scan-to-BIM repository\geomapi\geomapi\tools\machinelearningtools\__init__.py:184: RuntimeWarnin

Visualize some results

In [39]:
from matplotlib import pyplot as plt
import math
n=imgNodes[random.randint(  0,len(imgNodes))]
fig, axs = plt.subplots(1, 3,figsize=(30, 30))
fig.set_facecolor("white")
axs[0].imshow(iu.image_resize(n.resource,scale=1/1))
axs[1].imshow(iu.image_resize(n.colorImage2,scale=1/1))
axs[2].imshow(iu.image_resize(n.hha,scale=1/1))
plt.show()

IndexError: list index out of range

(optional) write class labels to file.

In [15]:
import shutil

shutil.rmtree(dir_img) if os.path.exists(dir_img) else None 
os.mkdir(dir_img) if not os.path.exists(dir_img) else None 
shutil.rmtree(dir_class) if os.path.exists(dir_class) else None 
os.mkdir(dir_class) if not os.path.exists(dir_class) else None                      
shutil.rmtree(dir_depth) if os.path.exists(dir_depth) else None 
os.mkdir(dir_depth) if not os.path.exists(dir_depth) else None    
shutil.rmtree(dir_hha) if os.path.exists(dir_hha) else None 
os.mkdir(dir_hha) if not os.path.exists(dir_hha) else None    
shutil.rmtree(dir_labels) if os.path.exists(dir_labels) else None 
os.mkdir(dir_labels) if not os.path.exists(dir_labels) else None  

In [22]:
for n in imgTrainingNodes:
    n.color=np.round(n.color*255,0) 

In [23]:
train_list = []
test_list = []

for i,n in enumerate(imgTrainingNodes):
    #save image
    cv2.imwrite(os.path.join(dir_img,n.name+'.png'), n.resource)
    #class image
    # classImg=iu.grb01_to_rgb255(n.colorImage2)
    cv2.imwrite(os.path.join(dir_class,n.name+'.png'), n.color)
    #depth
    cv2.imwrite(os.path.join(dir_depth,n.name+'.png'), n.depth)
    #hha
    cv2.imwrite(os.path.join(dir_hha,n.name+'.png'), n.hha)  
    #labels
    cv2.imwrite(os.path.join(dir_labels,n.name+'.png'), n.mask)

    if i%2 ==0:
        train_list.append(n.name)
    else:
        test_list.append(n.name)

def write_txt(path_list, names):
    with open(path_list, 'w') as f:
        [f.write(name+'\n') for name in names]

path_list = os.path.join(output_dir, 'train.txt')
write_txt(path_list, train_list)

path_list = os.path.join(output_dir, 'test.txt')
write_txt(path_list, test_list)

In [58]:
path='D:/Data/2023-01 NYU_v2/nyu_v2/depth'
import glob
test1 = cv2.imread('D:/Data/2023-01 Paestum/Research/shapeConv/depth/IMG_0609.png',cv2.IMREAD_GRAYSCALE)
print(test1.max())

test2 = cv2.imread('D:/Data/2023-01 NYU_v2/nyu_v2/depth/000117.png',cv2.IMREAD_GRAYSCALE)
print(test2.max())
# depth is different. we have inf and they have 14


59
38
